In [59]:
#imports
import time
import math 
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData, Table, Column, Float, String
from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from urllib.parse import urlparse
import re
import os

In [51]:
#initial testing with a single page before using loops later on. Using soup to find and scrape the values I was looking for 

# browser = Browser('edge')
#url = "https://www.canadiantire.ca/en/promotions/clearance.html?page=20"
#browser.visit(url)
#html = browser.html
#html_soup = soup(html, 'html.parser')

In [52]:
# testing area 

# product_info = html_soup.find_all(class_='nl-product__content')
# product_data1 = []

# for product in product_info:
#     product_name_element = product.find('div', class_='nl-product-card__title')
#     original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
#     clearance_price_element = product.find('span', class_='nl-price--total--red')
#     rating_element = product.find('div', class_='bv_text')
#     image_element = product.find('div', class_='nl-product-card__image-wrap')
#     anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
#     img_tag = product.find('img', attrs={'data-src': True})

#     product_name = product_name_element.text.strip() if product_name_element else 'N/A'
#     original_price = original_price_element.text.strip() if original_price_element else 'N/A'
#     clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
#     rating = rating_element.text.strip() if rating_element else None
#     product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
#     product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
#     product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
#     product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

#     if original_price == 'N/A':
#         alt_original_price_element = product.find('span', class_='nl-price--total')
#         original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

#     clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
#     original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

#     product_data1.append({
#         'Product Name': product_name,
#         'Original Price': original_price,
#         'Clearance Price': clearance_price,
#         'Rating': rating,
#         'Product Code': product_code,
#         'Link': product_link, 
#         'Image Link': product_image_link
#     })

# print(product_data1)

In [53]:
# testing area 

# df1 = pd.DataFrame(product_data1)
# df1.sample(24)
# df1['Percentage Off'] = round(((df1['Original Price'] - df1['Clearance Price']) / df1['Original Price']) * 100)
# df1.sample(24)

In [54]:
# testing area 

#setup urlparse to grab the product category from the image link. 

# def extract_category(link_url):
#     parsed_url = urlparse(link_url)
#     path_components = parsed_url.path.split('/')
#     category = '/'.join(path_components[3:4])
#     return category

# df1['Product Category'] = df1['Image Link'].apply(extract_category)

# df1

In [55]:
#establishing the base url from the clearance section

base_url = "https://www.canadiantire.ca/en/promotions/clearance.html?page="

#setting up some options for the chrome browser. Using the exp_option to click the allow location, and incognito mode to
#seperate the browers from my other open browsers. Also, repeat testing without cookies saved was helpful. 

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

#setup the Chromedriver. it was not working without the time delay, perhaps letting the page load fully.
service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(base_url)
time.sleep(5)
html_content = driver.page_source
html_soup = soup(html_content, 'html.parser')


# I wanted to dynamically determine the number of loops to use. I judged that total results, or total number of 
# products on sale, divided by the default number of products per page worked well. Using math.ceil to always round up. 

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'
total_results = int(re.search(r'\d+', total_results_text).group())
items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)


# creating a empty list to store data
product_data = []


#beginning the loop, adding time for the page to load, using a try except to handle a random survey pop-up. 

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

#gathering the ingredients for my soup. 

    html_content = driver.page_source
    html_soup = soup(html_content, 'html.parser')

    product_info = html_soup.find_all(class_='nl-product__content')

#another loop to grab all the info on the page. The first code chunk locates, the second chunk parses out. 
    for product in product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

#there were some products with prices in different places, so this fills in those spaces. 
        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

#adding some data cleaning and parsing to the loop to make it easier later on. 
        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Clearance Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver.quit()

#printing these as a way to check that the scraping was working
print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=115.0.5790.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0116A813+48355]
	(No symbol) [0x010FC4B1]
	(No symbol) [0x01005358]
	(No symbol) [0x00FED293]
	(No symbol) [0x0104E37B]
	(No symbol) [0x0105C473]
	(No symbol) [0x0104A536]
	(No symbol) [0x010282DC]
	(No symbol) [0x010293DD]
	GetHandleVerifier [0x013CAABD+2539405]
	GetHandleVerifier [0x0140A78F+2800735]
	GetHandleVerifier [0x0140456C+2775612]
	GetHandleVerifier [0x011F51E0+616112]
	(No symbol) [0x01105F8C]
	(No symbol) [0x01102328]
	(No symbol) [0x0110240B]
	(No symbol) [0x010F4FF7]
	BaseThreadInitThunk [0x76A67D59+25]
	RtlInitializeExceptionChain [0x77C7B79B+107]
	RtlClearBits [0x77C7B71F+191]


In [ ]:
# I wasn't actually interested in the image links as image links, but rather to parse out product category information, 
# which didn't seem to be present anywhere else on the page. I created a function that uses urllib.parse from urlparse
# to grab the information that I wanted. I was having trouble getting re to work here, and this was simpler.

def extract_category(link_url):
    parsed_url = urlparse(link_url)
    path_components = parsed_url.path.split('/')
    category = '/'.join(path_components[3:4])
    return category

In [ ]:
#creating the dataframe, creating a new column that calculates the clearance prices, applying my extract_category
clearance_df = pd.DataFrame(product_data)
clearance_df['Percentage Off'] = round(((clearance_df['Original Price'] - clearance_df['Clearance Price']) / clearance_df['Original Price']) * 100)
clearance_df['Product Category'] = clearance_df['Image Link'].apply(extract_category)

In [ ]:
# HOT-SALE scraping. Code is nearly idential to the above.
# scraping the sale section for additional data

base_url = "https://www.canadiantire.ca/en/promotions/hot-sale.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver2 = webdriver.Chrome(service=service)

driver2.get(base_url)
time.sleep(5)
html_content = driver2.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver2.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver2, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver2.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info = html_soup.find_all(class_='nl-product__content')

    for product in sale_product_info:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Sale Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver2.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

Total number of results: 1884
Total number of pages: 79


In [ ]:
sale_df = pd.DataFrame(sale_product_data)
sale_df['Percentage Off'] = round(((sale_df['Original Price'] - sale_df['Sale Price']) / sale_df['Original Price']) * 100)
sale_df['Product Category'] = sale_df['Image Link'].apply(extract_category)


In [ ]:
#RED-ALERT-DEALS scraping. Code is nearly idential to the above.
# scraping the sale section for additional data

base_url = "https://www.canadiantire.ca/en/promotions/red-alert-deals.html?page="

chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_experimental_option("prefs", {
    "profile.default_content_setting_values.geolocation": 1
})

service = Service("C:\Program Files\Common Files\ChromeDriver\chromedriver.exe")
driver3 = webdriver.Chrome(service=service)

driver3.get(base_url)
time.sleep(5)
html_content = driver3.page_source
html_soup = soup(html_content, 'html.parser')

total_results_element = html_soup.find('span', class_='nl-filters__results')
total_results_text = total_results_element.text if total_results_element else '0'

total_results = int(re.search(r'\d+', total_results_text).group())

items_per_page = 24
total_pages = math.ceil(total_results / items_per_page)

sale_product_data_2 = []

for page_number in range(1, total_pages + 1):
    url = base_url + str(page_number)
    driver3.get(url)
    time.sleep(5)

    try:
        not_right_now_button = WebDriverWait(driver3, 10).until(
            EC.element_to_be_clickable((By.ID, "kplDeclineButton"))
        )
        not_right_now_button.click()
    except:
        pass

    html_content = driver3.page_source
    html_soup = soup(html_content, 'html.parser')

    sale_product_info2 = html_soup.find_all(class_='nl-product__content')

    for product in sale_product_info2:
        product_name_element = product.find('div', class_='nl-product-card__title')
        original_price_element = product.find('s', attrs={'aria-hidden': 'true'})
        clearance_price_element = product.find('span', class_='nl-price--total--red')
        rating_element = product.find('div', class_='bv_text')
        image_element = product.find('div', class_='nl-product-card__image-wrap')
        anchor_tag = product.find('a', class_='nl-product-card__no-button', href=True)
        img_tag = product.find('img', attrs={'data-src': True})

        product_name = product_name_element.text.strip() if product_name_element else 'N/A'
        original_price = original_price_element.text.strip() if original_price_element else 'N/A'
        clearance_price = clearance_price_element.text.strip() if clearance_price_element else 'N/A'
        rating = rating_element.text.strip() if rating_element else None
        product_code_element = product.find('p', class_='nl-product__code', attrs={'aria-hidden': 'true'})
        product_code = product_code_element.get_text(strip=True).lstrip('#') if product_code_element else 'N/A'
        product_link = "https://www.canadiantire.ca" + anchor_tag['href'] if anchor_tag is not None else 'N/A'
        product_image_link = img_tag['data-src'] if img_tag and 'data-src' in img_tag.attrs else 'N/A'

        if original_price == 'N/A':
            alt_original_price_element = product.find('span', class_='nl-price--total')
            original_price = alt_original_price_element.text.strip() if alt_original_price_element else 'N/A'

        clearance_price = float(clearance_price.replace('Each', '').replace('$', '').replace(',', '')) if clearance_price != 'N/A' else None
        original_price = float(original_price.replace('Each', '').replace('$', '').replace(',', '')) if original_price != 'N/A' else None

        sale_product_data_2.append({
            'Product Name': product_name,
            'Original Price': original_price,
            'Sale Price': clearance_price,
            'Rating': rating,
            'Product Code': product_code,
            'Link': product_link, 
            'Image Link': product_image_link
        })

driver3.quit()

print(f"Total number of results: {total_results}")
print(f"Total number of pages: {total_pages}")

Total number of results: 497
Total number of pages: 21


In [ ]:
sale_df2 = pd.DataFrame(sale_product_data_2)
sale_df2['Percentage Off'] = round(((sale_df2['Original Price'] - sale_df2['Sale Price']) / sale_df2['Original Price']) * 100)
sale_df2['Product Category'] = sale_df2['Image Link'].apply(extract_category)

In [ ]:
#checking
print(len(clearance_df))
clearance_df.sample(5)

1154


,Product Name,Original Price,Clearance Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
269,"Bluehive Mini USB Charger and Sync Cable, 3-ft",6.99,2.93,4.7,035-0369X,https://www.canadiantire.ca/en/pdp/bluehive-mi...,https://media-www.canadiantire.ca/product/auto...,58.0,car-care-accessories
39,Solignum Marine Interior/Exterior Alkyd Enamel...,NaN,60.99,4.0,048-9705X,https://www.canadiantire.ca/en/pdp/solignum-ma...,https://media-www.canadiantire.ca/product/fixi...,NaN,paint
440,"Bow ABS Drainage Pipe, 3-in x 10-ft",49.99,32.93,5.0,063-2307-0,https://www.canadiantire.ca/en/pdp/bow-abs-dra...,https://media-www.canadiantire.ca/product/fixi...,34.0,plumbing
126,CANVAS Performance Cotton Absorbent Wash Towel...,NaN,7.99,4.0,063-6243X,https://www.canadiantire.ca/en/pdp/canvas-perf...,https://media-www.canadiantire.ca/product/livi...,NaN,home-decor
408,"Armor Coat Multi-Purpose Super Glue, Indoor/Ou...",3.99,2.43,3.3,067-0342-8,https://www.canadiantire.ca/en/pdp/armor-coat-...,https://media-www.canadiantire.ca/product/fixi...,39.0,paint


In [ ]:
#checking
print(len(sale_df))
sale_df.sample(5)

1884


,Product Name,Original Price,Sale Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
103,MASTER Chef Grill Turismo Wood Pellet Grill & ...,499.99,449.99,3.6,085-3185-0,https://www.canadiantire.ca/en/pdp/master-chef...,https://media-www.canadiantire.ca/product/seas...,10.0,backyard-living
321,PowerBilt TPX CX4 4-Wheel Golf Cart,249.99,149.99,4.5,086-0603-2,https://www.canadiantire.ca/en/pdp/powerbilt-t...,https://media-www.canadiantire.ca/product/play...,40.0,team-sports-and-golf
1624,Nutro Grain Free Cuts in Gravy Chicken & Beef ...,23.99,21.59,5.0,142-5975-2,https://www.canadiantire.ca/en/pdp/nutro-grain...,https://media-www.canadiantire.ca/product/livi...,10.0,pet-care
95,BISSELL® Pet Hair Eraser Pet Pro Corded Stick ...,359.99,179.99,3.9,043-8600-2,https://www.canadiantire.ca/en/pdp/bissell-pet...,https://media-www.canadiantire.ca/product/livi...,50.0,cleaning
1356,Pine-Sol Spring Blossom Multi-Surface Cleaner ...,6.49,5.84,4.1,153-0512-8,https://www.canadiantire.ca/en/pdp/pine-sol-sp...,https://media-www.canadiantire.ca/product/livi...,10.0,cleaning


In [ ]:
#checking
print(len(sale_df2))
sale_df2.sample(5)

497


,Product Name,Original Price,Sale Price,Rating,Product Code,Link,Image Link,Percentage Off,Product Category
93,Mobil 1™ Extended Performance 5W20 Synthetic E...,74.99,40.99,4.8,028-9462-2,https://www.canadiantire.ca/en/pdp/mobil-1-ext...,https://media-www.canadiantire.ca/product/auto...,45.0,auto-maintenance
73,"Heritage The Rock Frying Pan Non-stick, Dishwa...",139.99,29.99,4.5,142-7073-6,https://www.canadiantire.ca/en/pdp/heritage-th...,https://media-www.canadiantire.ca/product/livi...,79.0,kitchen
297,Arm & Hammer AbsorbX Lightweight Multi-Cat Clu...,27.99,26.09,3.4,142-9425-8,https://www.canadiantire.ca/en/pdp/arm-hammer-...,https://media-www.canadiantire.ca/product/livi...,7.0,pet-care
77,Glad Extra-Large Easy-Tie Flap Indoor/Outdoor ...,12.99,10.39,4.7,042-9715-6,https://www.canadiantire.ca/en/pdp/glad-extra-...,https://media-www.canadiantire.ca/product/livi...,20.0,cleaning
336,SIMONIZ Platinum Microfibre Chenille Wash Mitt...,9.99,8.49,4.0,039-7065-4,https://www.canadiantire.ca/en/pdp/simoniz-pla...,https://media-www.canadiantire.ca/product/auto...,15.0,car-care-accessories


In [ ]:
#checking
clearance_df.dtypes

Product Name         object
Original Price      float64
Clearance Price     float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [ ]:
#checking
sale_df.dtypes

Product Name         object
Original Price      float64
Sale Price          float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [ ]:
#checking
sale_df2.dtypes

Product Name         object
Original Price      float64
Sale Price          float64
Rating               object
Product Code         object
Link                 object
Image Link           object
Percentage Off      float64
Product Category     object
dtype: object

In [ ]:
#no longer needed, I just used them to parse out the product type
clearance_df.drop('Image Link', axis=1, inplace=True)
sale_df.drop('Image Link', axis=1, inplace=True)
sale_df2.drop('Image Link', axis=1, inplace=True)

In [ ]:
#reordering columns so they make more sense to me. 
clearance_column_order = ['Product Name',
                    'Original Price',
                    'Clearance Price',
                    'Percentage Off',
                    'Product Category',
                    'Product Code',
                    'Rating',
                    'Link']

sale_column_order = ['Product Name',
                    'Original Price',
                    'Sale Price',
                    'Percentage Off',
                    'Product Category',
                    'Product Code',
                    'Rating',
                    'Link']

clearance_df = clearance_df.loc[:, clearance_column_order]
sale_df = sale_df.loc[:, sale_column_order]
sale_df2 = sale_df2.loc[:, sale_column_order]

In [ ]:
#change astype in case I want to use this later
clearance_df['Rating'] = clearance_df['Rating'].astype(float)
sale_df['Rating'] = sale_df['Rating'].astype(float)
sale_df2['Rating'] = sale_df2['Rating'].astype(float)

In [ ]:
#checking
clearance_df.sample(5)

,Product Name,Original Price,Clearance Price,Percentage Off,Product Category,Product Code,Rating,Link
52,Thermacell 6.5-Hour DEET-Free Radius Zone Mosq...,69.99,34.93,50.0,gardening,059-1286-6,3.6,https://www.canadiantire.ca/en/pdp/thermacell-...
696,"No Leak Engine Oil Stop Leak, 473-mL",11.99,6.93,42.0,auto-maintenance,038-0221-6,3.3,https://www.canadiantire.ca/en/pdp/no-leak-eng...
450,"Alpena LED Light, 8-ft",59.99,6.93,88.0,car-care-accessories,020-3787X,1.0,https://www.canadiantire.ca/en/pdp/alpena-led-...
352,Atlas 3-Bolt Lawn Mower Blade Adapter Kit,24.99,2.93,88.0,outdoor-tools,060-7032-8,0.0,https://www.canadiantire.ca/en/pdp/atlas-3-bol...
512,Dupli-Color Perfect Match Premium Acrylic Lacq...,21.99,3.93,82.0,car-care-accessories,047-2676-0,2.5,https://www.canadiantire.ca/en/pdp/dupli-color...


In [ ]:
#checking
sale_df.sample(5)

,Product Name,Original Price,Sale Price,Percentage Off,Product Category,Product Code,Rating,Link
488,SIMONIZ Microfibre Multi-Purpose Edgeless Towe...,23.99,17.99,25.0,car-care-accessories,039-6631-2,4.0,https://www.canadiantire.ca/en/pdp/simoniz-mic...
1628,"Thrush Universal Welded Muffler, 17657",99.99,84.99,15.0,heavy-auto-parts,112-8358-6,0.0,https://www.canadiantire.ca/en/pdp/thrush-univ...
96,Summer Waves® Round Quick Set Inflatable Pool ...,289.99,269.99,7.0,backyard-fun,081-3081-4,2.9,https://www.canadiantire.ca/en/pdp/summer-wave...
1020,"Raleigh Steel Bike Padlock w/2 Keys & Velcro, ...",38.99,33.09,15.0,cycling,073-3055-8,4.7,https://www.canadiantire.ca/en/pdp/raleigh-ste...
1278,"MotoMaster 12-Ply Trailer Tire, ST235/80R16",219.99,197.99,10.0,automotive-outdoor-adventure,040-3170-0,0.0,https://www.canadiantire.ca/en/pdp/motomaster-...


In [ ]:
#checking
sale_df2.sample(5)

,Product Name,Original Price,Sale Price,Percentage Off,Product Category,Product Code,Rating,Link
237,NOMA PAR20 E26 Base Dimmable LED Flood Light B...,31.99,9.59,70.0,electrical,052-2386-6,4.0,https://www.canadiantire.ca/en/pdp/noma-par20-...
73,"Heritage The Rock Frying Pan Non-stick, Dishwa...",139.99,29.99,79.0,kitchen,142-7073-6,4.5,https://www.canadiantire.ca/en/pdp/heritage-th...
286,MotoMaster Full Synthetic Multi-Vehicle Low Vi...,49.99,44.99,10.0,auto-maintenance,028-1575-2,4.4,https://www.canadiantire.ca/en/pdp/motomaster-...
445,Thermos Vacuum Insulated Stainless Steel Porta...,49.99,39.99,20.0,kitchen,142-7378-2,0.0,https://www.canadiantire.ca/en/pdp/thermos-vac...
199,Greenies Original Large Oral Care Dental Dog T...,19.99,17.99,10.0,pet-care,142-5665-0,4.5,https://www.canadiantire.ca/en/pdp/greenies-or...


In [ ]:
#Removing characters that were causing sql import issues. 

clearance_df["Product Name"] = clearance_df["Product Name"].str.replace("'", '')
clearance_df["Product Name"] = clearance_df["Product Name"].str.replace("Â®", '')
sale_df["Product Name"] = sale_df["Product Name"].str.replace("'", '')
sale_df["Product Name"] = sale_df["Product Name"].str.replace("Â®", '')
sale_df2["Product Name"] = sale_df2["Product Name"].str.replace("'", '')
sale_df2["Product Name"] = sale_df2["Product Name"].str.replace("Â®", '')

In [ ]:
clearance_df.to_csv("./data/clearance.csv", index=False)
sale_df.to_csv("./data/sale.csv", index=False)
sale_df2.to_csv("./data/sale2.csv", index=False)

In [ ]:
#just in case I only want to work with all the sale products, not seperated by specific promotion
allsales_df = pd.concat([sale_df, sale_df2], ignore_index=True)
allsales_df.to_csv("./data/allsales.csv", index=False)

allproducts_df = pd.concat([clearance_df, sale_df, sale_df2], ignore_index=True)
allproducts_df.to_csv("./data/allproducts_df.csv", index=False)

In [ ]:
# I was thinking about saving the files with a datatime in case I wanted to do historical analysis at a later date. 
from datetime import datetime

current_date = datetime.now().strftime("%Y-%m-%d")

clearance_df.to_csv(f"./data/clearance_{current_date}.csv", index=False)
sale_df2.to_csv(f"./data/sale2_{current_date}.csv", index=False)
allsales_df.to_csv(f"./data/allsales_{current_date}.csv", index=False)
allproducts_df.to_csv(f"./data/allproducts_{current_date}.csv", index=False)

In [61]:
# # Connection details for uploading to server
# username = os.environ.get('DB_USERNAME')
# password = os.environ.get('DB_PASSWORD')
# host = os.environ.get('DB_HOST')
# database = os.environ.get('DB_NAME')

# # Create the database connection URL
# url = f'postgresql://{username}:{password}@{host}/{database}'

# engine = create_engine(url)

In [57]:
# # Write the DataFrames to the database tables
# clearance_table = 'clearance'
# clearance_data = pd.read_csv("./data/clearance.csv")
# clearance_data.reset_index(drop=True, inplace=True)
# clearance_data.to_sql(clearance_table, engine, if_exists='replace', index=False)

154

In [58]:
# allsales_table = 'sales'
# allsales_data = pd.read_csv("./data/allsales.csv")
# allsales_data.reset_index(drop=True, inplace=True)
# allsales_data.to_sql(allsales_table, engine, if_exists='replace', index=False)

381